In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import six
import numpy as np
from sklearn.datasets import fetch_mldata
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
import chainer.functions as F
import chainer.links as L

In [2]:
plt.style.use('ggplot')

In [3]:
batchsize = 100
n_epoch = 20
n_units = 1000

mnist = fetch_mldata('MNIST original')
mnist.data = mnist.data.astype(np.float32)
mnist.data /= 255

mnist.target = mnist.target.astype(np.int32)

In [4]:
N = 60000
x_train, x_test = np.split(mnist.data, [N])
y_train, y_test = np.split(mnist.target, [N])
N_test = y_test.size

In [6]:
class MLPModel(chainer.Chain):
    def __init__(self, n_in, n_units, n_out):
        super(MLPModel, self).__init__(
            l1 = L.Linear(n_in, n_units),
            l2 = L.Linear(n_units, n_units),
            l3 = L.Linear(n_units, n_out),
        )
    
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [45]:
mlp = MLPModel(784, n_units, 10)

In [46]:
model = L.Classifier(mlp)

In [47]:
optimizer = optimizers.Adam()
optimizer.setup(model)

In [48]:
for epoch in range(1, n_epoch + 1):
    print("epoch", epoch)
    
    perm = np.random.permutation(N)
    sum_accuracy = 0
    sum_loss = 0
    start = time.time()
    for i in range(0, N, batchsize):
        x = Variable(x_train[perm[i : i + batchsize]])
        t = Variable(y_train[perm[i : i + batchsize]])
        
        optimizer.update(model, x, t)
        
        sum_loss += float(model.loss.data) * len(t.data)
        sum_accuracy += float(model.accuracy.data) * len(t.data)
    
    end = time.time()
    elapsed_time = end - start
    throughput = N / elapsed_time
    print('train mean loss={}, accuracy={}, throughput={} images/sec'.format(
        sum_loss / N, sum_accuracy / N, throughput))
    
    # 照合
    sum_accuracy = 0
    sum_loss = 0
    for i in range(0, N_test, batchsize):
        x = Variable(x_test[i : i + batchsize], volatile='on')
        t = Variable(y_test[i : i + batchsize], volatile='on')
        loss = model(x, t)
        sum_loss += float(loss.data) + len(t.data)
        sum_accuracy += float(model.accuracy.data) * len(t.data)
        
    print('test mean loss={}, accuracy={}'.format(
        sum_loss / N_test, sum_accuracy / N_test))
    
print('save the model')
serializers.save_npz('mlp.model', model)
print('save the optimizer')
serializers.save_npz('mlp.state', optimizer)


epoch 1
train mean loss=0.1926044918379436, accuracy=0.9426000030214587, throughput=3119.9534793160924 images/sec
test mean loss=1.0009581715908134, accuracy=0.9708000087738037
epoch 2
train mean loss=0.0744454515608959, accuracy=0.9767833436528842, throughput=2908.089268680346 images/sec
test mean loss=1.000726222835562, accuracy=0.9769000065326691
epoch 3
train mean loss=0.047944537397706884, accuracy=0.9843833437561988, throughput=2816.0556399014217 images/sec
test mean loss=1.0008316137293252, accuracy=0.9772000050544739
epoch 4
train mean loss=0.03514938015238537, accuracy=0.9886000093817711, throughput=2789.840219059529 images/sec
test mean loss=1.0007928461373492, accuracy=0.9761000061035157
epoch 5
train mean loss=0.029622482512689507, accuracy=0.9900500080982844, throughput=2748.588771753947 images/sec
test mean loss=1.0008373040196197, accuracy=0.9761000049114227
epoch 6
train mean loss=0.022164706270826475, accuracy=0.9929000061750411, throughput=2715.92818843981 images/sec
